<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/803/best_dataset_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
import pandas as pd
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset('HamdanXI/fb_labeled')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4062 [00:00<?, ? examples/s]

In [18]:
def extract_ids(example):
    parts = example['file_name'].split('_')
    example['EpId'] = parts[1].lstrip('0')
    example['ClipId'] = parts[2].lstrip('0').split('.')[0]  # Remove the file extension
    return example

# Apply the function to add EpId and ClipId to the dataset
dataset_with_ids = dataset['train'].map(extract_ids)


output_dataset = load_dataset('csv', data_files="output.csv")

In [21]:
from datasets import concatenate_datasets, Dataset, ClassLabel, Value

dataset_with_ids_new = dataset_with_ids.cast_column('EpId', Value('string'))
dataset_with_ids_new = dataset_with_ids_new.cast_column('ClipId', Value('string'))

output_dataset_new = output_dataset.cast_column('EpId', Value('string'))
output_dataset_new = output_dataset_new.cast_column('ClipId', Value('string'))

# Concatenate datasets
new_dataset = concatenate_datasets([dataset_with_ids_new, output_dataset_new['train']])

In [22]:
new_dataset

Dataset({
    features: ['file_name', 'audio', 'EpId', 'ClipId', 'Output'],
    num_rows: 9539
})

In [25]:
from huggingface_hub import notebook_login

notebook_login()

In [26]:
new_dataset.push_to_hub("HamdanXI/fb_labeled_v4")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/4770 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/48 [00:00<?, ?ba/s]

Map:   0%|          | 0/4769 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/48 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/HamdanXI/fb_labeled_v4/commit/b4831137f3f89d36a9693883c9dc5c3a7fcd5cb6', commit_message='Upload dataset', commit_description='', oid='b4831137f3f89d36a9693883c9dc5c3a7fcd5cb6', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
new_dataset

Dataset({
    features: ['file_name', 'audio', 'EpId', 'ClipId', 'Output'],
    num_rows: 9539
})

In [30]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("HamdanXI/fb_labeled_v4")

In [35]:
from datasets import load_dataset

# Assume we are working with the 'train' split
data = dataset['train']

# Create dictionaries to map (EpId, ClipId) to 'audio' and 'Output'
audio_map = {}
output_map = {}

# Populate the dictionaries
for example in data:
    key = (example['EpId'], example['ClipId'])
    if example['audio'] is not None:
        audio_map[key] = example['audio']
    if example['Output'] is not None:
        output_map[key] = example['Output']

# Function to apply mappings
def apply_mappings(example):
    key = (example['EpId'], example['ClipId'])
    example['audio'] = audio_map.get(key, None)
    example['Output'] = output_map.get(key, None)
    return example

# Apply the map function to populate the merged dataset
merged_dataset = data.map(apply_mappings)

Map:   0%|          | 0/9539 [00:00<?, ? examples/s]

In [36]:
merged_dataset

Dataset({
    features: ['file_name', 'audio', 'EpId', 'ClipId', 'Output'],
    num_rows: 9539
})

In [37]:
merged_dataset[0]

{'file_name': 'FluencyBank_081_129.wav',
 'audio': {'path': None,
  'array': array([-0.00918579, -0.00836182, -0.00769043, ..., -0.00045776,
         -0.00085449, -0.00268555]),
  'sampling_rate': 16000},
 'EpId': '81',
 'ClipId': '129',
 'Output': 8}

In [38]:
merged_dataset.push_to_hub("HamdanXI/fb_labeled_v5")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/4770 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/48 [00:00<?, ?ba/s]

Map:   0%|          | 0/4769 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/48 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/HamdanXI/fb_labeled_v5/commit/5dda536f6cee5da1e362306376e26c176dec9869', commit_message='Upload dataset', commit_description='', oid='5dda536f6cee5da1e362306376e26c176dec9869', pr_url=None, pr_revision=None, pr_num=None)